# ZO (Oats) Overnight Algo

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import yfinance as yf
import bt
from ib_insync import *
import nest_asyncio
import logging
import datetime
import schedule
import time

In [6]:
nest_asyncio.apply()
# Connect to TWS (or gateway)
ib = IB()
ib.connect('127.0.0.1', 7496, clientId=420) #7497-Paper Trading, 7496-Live trading

<IB connected to 127.0.0.1:7496 clientId=420>

In [7]:
#Define ES contract
contract = Future('ZO', '202505', 'CBOT')

# Qualify the contract (ensures IB recognizes it)
ib.qualifyContracts(contract)

[Future(conId=630781096, symbol='ZO', lastTradeDateOrContractMonth='20250514', multiplier='5000', exchange='CBOT', currency='USD', localSymbol='ZOK5', tradingClass='ZO')]

## ZO Oats Overnight Algorithm

In [8]:
# Production Version

# Initialize global flag
last_buy_trade_date = None
last_sell_trade_date = None

def execute_buy_trade():
    global last_buy_trade_date
    now = datetime.datetime.now()
    today = datetime.datetime.now()
    buy_days = [0,1,2,3,4]  
    trade_time = datetime.time(14, 19, 56) #14, 19, 56

    # Check if day and time matches criteria above
    if now.weekday() in buy_days and \
       now.time().hour == trade_time.hour and \
       now.time().minute == trade_time.minute and \
       now.time().second == trade_time.second:
       
        # Check if buy trade already been executed today
        if last_buy_trade_date == now.date():
            print("Buy Trade already executed today.")
            return

        # Define contract & order
        contract = Future('ZO', '202505', 'CBOT')
        order = MarketOrder("Buy", 1)  

        # Place trade
        ib.placeOrder(contract, order)
        print("Buy Order Placed.", today.strftime('%Y-%m-%d %H:%M:%S'))

        # Log trade
        logging.info(f"Executed Trade: {today.strftime('%Y-%m-%d %H:%M:%S')} - Buy 3 ZO")
        
        # Update the last trade date to avoid duplicate trades for the day
        last_buy_trade_date = now.date()
        return

    print("No BUY Condition Met.")

def execute_sell_trade():
    global last_sell_trade_date
    now = datetime.datetime.now()
    today = datetime.datetime.now()
    buy_days = [0,1,2,3,4]  
    trade_time = datetime.time(9, 29, 56) #9 29 56

    # Check if day and time matches criteria above
    if now.weekday() in buy_days and \
       now.time().hour == trade_time.hour and \
       now.time().minute == trade_time.minute and \
       now.time().second == trade_time.second:
       
        # Check if trade already been executed today
        if last_sell_trade_date == now.date():
            print("Sell Trade Already executed today.")
            return

        # Define contract & order
        contract = Future('ZO', '202505', 'CBOT')
        order = MarketOrder("Sell", 3)  

        # Place trade
        ib.placeOrder(contract, order)
        print("Sell Order Placed.", today.strftime('%Y-%m-%d %H:%M:%S'))

        # Log trade
        logging.info(f"Executed Trade: {today.strftime('%Y-%m-%d %H:%M:%S')} - Sell 3 ZO")
        
        # Update the last trade date to avoid duplicate trades for the day
        last_sell_trade_date = now.date()
        return

    print("No SELL Condition Met.")

schedule.every().day.at("14:19:56").do(execute_buy_trade) #14:19:56 
schedule.every().day.at("09:29:56").do(execute_sell_trade) #09:29:58

# Keep scheduler running
while True:
    schedule.run_pending()
    time.sleep(0.50)

Buy Order Placed. 2025-02-26 14:19:56
Buy Trade already executed today.


KeyboardInterrupt: 